# Multinomial Logistic Regression (MLR)

- Multinomial logistic regression, sometimes also called **softmax regression**
or **maximum entropy classification**, is a method of extending logistic regression,
which by default only works for binary classification problems, to multi-class
classification problems.

## Set up

- We assume our classes are again labeled as integers, but rather than 0 and 1, if
we have $K$ total classes, we
will call them classes $1$ through $K$.

- In our training data, though we could think of our new target $y$ values as a value
also between 1 and $K$, we will turn $y$ into a vector of zeros and ones where the 
correct class has a value of 1 and the other entries have a value of 0 (called a 
**one-hot vector**.

## Example

Suppose we wish to predict a student's grade on a test from the amount of time
they spend studying for the test (in hours) and the number of absences they had 
in class.  Assume the grade predicted will be A, B, or C (we're going to be optimistic
about outcomes!)

Our classes could be A=1, B=2, C=3 (though this particular assignment of grades
to the class labels is arbitrary).  Imagine our data set looks like this:

In [4]:
import pandas as pd

data = [['Alice', 8, 1, 'A'], ['Bob', 5, 2, 'B'], ['Carol', 6, 3, 'C'], ['Dan', 4, 1, 'B']]
df = pd.DataFrame(data, columns=['Name', 'Hours', 'Absences', 'Grade'])
df

,Name,Hours,Absences,Grade
0,Alice,8,1,A
1,Bob,5,2,B
2,Carol,6,3,C
3,Dan,4,1,B


In [14]:
# Encode a one-hot vector using pandas "get_dummies()" function:

one_hot = pd.get_dummies(df['Grade'])
one_hot

# Each row is a "one-hot vector" encoding the class for the
# equivalent row in df.

,A,B,C
0,1,0,0
1,0,1,0
2,0,0,1
3,0,1,0


In [15]:
# For example, Bob's vector (row 1 above) is:
one_hot.to_numpy()[1]

array([0, 1, 0], dtype=uint8)

Multinomial logistic regression also **predicts** vectors.
While the input vectors to the algorithm are one-hot vectors,
the output of the model that multinomial logistic regression creates will be **probability** vectors, in that
the entries are interpreted to be probabilities (each between 0 and 1)
that collectively sum to 1 (so they form a probability distribution).

For example, suppose a student studies for 5 hours and has missed 3 classes.
A probability vector might look like $[0.1, 0.45, 0.45]$, indicating our model is predicting
that this student has a 10% chance of getting an "A", and a 45% chance each of getting
a B or a C on this test.

Note how the probabilities sum to 1.

**In summary**, whenever we see a $\boldsymbol{y}$ or $\boldsymbol{\hat{y}}$ in this
algorithm, remember it's a vector now (with length equal to $K$, the number of classes).

## The softmax function

The softmax function is a generalization of the sigmoid/logistic function, which we
denoted as $\sigma(z) = \dfrac{1}{1+e^{-z}}$.  We will need this function when we
define the model for multinomial logistic regression.

Because we will be using the $e^x$ function a lot, and to make the equations easier
to read, we will often use the notation $\exp(x) = e^x$.

The softmax function generalizes the sigmoid function to take a vector as input and also produces
a vector as output.

Suppose we have a vector $\boldsymbol{z}$ with length $K$.  So $\boldsymbol{z}=
[z_1, z_2, \ldots, z_K]$

Then,

$$\begin{align*}
\text{softmax}(\boldsymbol{z}) &= \dfrac{\exp(z)}{\displaystyle \sum_{k=1}^K \exp(z_k)}
=\left[ \dfrac{\exp(z_1)}{\displaystyle \sum_{k=1}^K \exp(z_k)}, 
\dfrac{\exp(z_2)}{\displaystyle \sum_{k=1}^K \exp(z_k)}, \ldots, 
\dfrac{\exp(z_K)}{\displaystyle \sum_{k=1}^K \exp(z_k)} \right] \\
 &= \dfrac{1}{\displaystyle \sum_{k=1}^K \exp(z_k)}
 \left[ \exp(z_1), \exp(z_2), \ldots, \exp(z_K) \right]
\end{align*}$$

This may not look very similar to $\dfrac{1}{1 + e^{-z}}$, but it's easier to see
if we remember the property
$$\sigma(z) = \dfrac{1}{1+e^{-z}} = \dfrac{e^z}{e^z+1} = \dfrac{\exp(z)}{\exp(z) + 1}.$$

In [21]:
# Example of softmax

from scipy.special import softmax
z = [0.6, 1.1, -1.5, 1.2, 3.2, -1.1]

softmax(z)

array([0.05482541, 0.09039182, 0.00671372, 0.09989841, 0.73815494,
       0.0100157 ])

In [22]:
# Entries sum to 1, so this is a probability distribution.

sum(softmax(z))

0.9999999999999999

## Model for multinomial logistic regression

In (regular) logistic regression, recall we had   $n$ features and $m$ training examples.
 
We also had a 
single $w$ vector:
$\boldsymbol{w} = [w_0, w_1, \ldots, w_n]^T$, and our model was 

$$f_\boldsymbol{w}(x) = \frac{1}{1 + e^{-w\cdot x}}$$

which predicted the probability of the example $x$ being in the "1" class (or
positive class).

Recall that the initial number $w_0$ takes the place of the old variable $b$.

In multinomial logistic regression, we will now have **separate** $w$ vectors for each class, from 1 to $K$.  So we will
have a $\boldsymbol{w}_1$ vector, and a $\boldsymbol{w}_2$ vector, all the way up to
a $\boldsymbol{w}_K$ vector.

Our model then becomes:

$$f_\boldsymbol{w_1, w_2, \ldots, w_K}(x) = [\hat{y}_1, \hat{y}_2, \ldots, \hat{y}_K]^T$$

where each $\hat{y}_k$ is defined by 

$$\hat{y}_k = \dfrac{\exp(\boldsymbol{w}_k \cdot x)}{\displaystyle \sum_{j=1}^K \exp(\boldsymbol{w}_j \cdot x)}$$

In other words, our model is outputting a vector $\boldsymbol{\hat{y}}$, where each
component $\hat{y}_k$ is determined by the equation above.

### With matrices

We will sometimes put all of these $\boldsymbol{w}$ vectors into a $\boldsymbol{W}$ matrix, where each
row of the matrix is one of our $\boldsymbol{w}$ vectors:

$$\begin{bmatrix}
\leftarrow & \boldsymbol{w_1} & \rightarrow\\
\leftarrow & \boldsymbol{w_2} & \rightarrow\\
 & \vdots & \\
\leftarrow & \boldsymbol{w_K} & \rightarrow\\
\end{bmatrix}$$

Note that this matrix has $K$ rows and $n+1$ columns.

We can then calculate $\boldsymbol{\hat{y}}$ all at once by:

$$\boldsymbol{\hat{y}} = \text{softmax}(\boldsymbol{W}x)$$

## Loss function

We define our loss function for MLR!  Remember the loss function is how much we
penalize each individual training example.

Recall the loss function for binary logistic regression:

$$L \left( f_w(x^{(i)}), y^{(i)} \right) = 
        -y^{(i)}\log\left( f_w(x^{(i)}) \right)-
        (1-y^{(i)})\log\left( 1-f_w(x^{(i)}) \right)$$
        
I'm going to simplify this and rewrite $f_w(x^{(i)})$ as just $\hat{y}$ and remove
all the $(i)$ parts.  So just assume $y$ is a target value for a single training example 
$x$ and $\hat{y}$ is the prediction: $f_w(x)$:

$$L \left( \hat{y}, y \right) = 
        -y\log\left( \hat{y} \right)-
        (1-y)\log\left( 1-\hat{y} \right)$$
        
Remember the interpretation of this formula:  since $y$ is either 0 or 1, 
either the first or second term of this expression will be zero and drop out.
This is because either $y$ is 0 (first term drops out) or $1-y$ is zero (second 
term drops out).

This formula is sometimes called the **cross-entropy loss function**.

### Extending this to MLR

Notice how the formula above is basically adding up two terms, each of which is 
of the form $-\text{something}\log(\hat{\text{something}})$.

We extend this idea, now remembering that $\boldsymbol{y}$ and $\boldsymbol{\hat{y}}$
are vectors:

$$L( \boldsymbol{\hat{y}}, \boldsymbol{y}) = -\sum_{k=1}^K y_k \log \hat{y}_k$$

A similar idea that happens above also happens here: all but one of these summations
will drop out because $\boldsymbol{y}$ is a one-hot vector.  So only one of the $y_k$
terms above is 1; all the rest are zeros.  Let's call the $y_k$ that **is** 1 $y_c$
($c$ standing for "correct," meaning the "correct class"):

$$L( \boldsymbol{\hat{y}}, \boldsymbol{y}) = -y_c \log \hat{y}_c = -\log \hat{y}_c$$

And now we can substitute in our formula above for each component of a 
$\hat{\boldsymbol{y}}$ vector:

$$L( \boldsymbol{\hat{y}}, \boldsymbol{y}) = -\log \hat{y}_c=
-\log \dfrac{\exp(\boldsymbol{w}_c \cdot x)}{\displaystyle \sum_{j=1}^K \exp(\boldsymbol{w}_j \cdot x)}$$

        

## Cost function

We define the cost function $J$ in the same way we always have:
as the average of the loss function calculated across our entire training set:

$$J(\boldsymbol{w_1, w_2, \ldots, w_K}) = J(\boldsymbol{W}) = \frac{1}{m}\sum_{i=1}^m L \left( 
    f(x^{(i)}), \boldsymbol{y}^{(i)} \right) = \frac{1}{m}\sum_{i=1}^m L \left( 
    \hat{\boldsymbol{y}}^{(i)}, \boldsymbol{y}^{(i)} \right)$$
    
Where the loss function $L$ is defined as above.

## Gradient descent

As always, we will use gradient descent to find the best collection of vectors
$\boldsymbol{w_1, w_2, \ldots, w_K}$.

We start by finding the (partial) derivative of the cost function with respect to an individual
number within a vector $w_k$.  Recall that for (regular) logistic regression, this
calculation was:

$$\dfrac{\partial}{\partial w_j} J(\boldsymbol{w}) =  \frac{1}{m} \sum_{i=1}^m  \left( f_{w}(x^{(i)}) - y^{(i)} \right)  x^{(i)}_j = \frac{1}{m} \sum_{i=1}^m  \left( \hat{y}^{(i)} - y^{(i)} \right)  x^{(i)}_j$$

Remember that the equation above is actually **multiple** equations, one per parameter $w_j$.

In MLR, the major changes are that $y$ and $\hat{y}$ are now vectors, and we now have
multiple $w$ vectors (that we will sometimes put into a $W$ matrix).

With these changes, there is now one equation per **entry** in each $w$ vector.
The equation itself actually doesn't change that much.  For each class $k$, we have 
a vector $w_k$, and the $i$'th entry in $w_k$ we will denote by $w_{k,j}$.  The
equation for the partial derivative of $J$ with respect to $w_{k,j}$ is:

$$\dfrac{\partial}{\partial w_{k,j}} J(\boldsymbol{w_1, w_2, \ldots, w_K}) =  
\frac{1}{m} \sum_{i=1}^m \left( \boldsymbol{\hat{y}}_k^{(i)} - \boldsymbol{y}^{(i)}_k \right)x^{(i)}_j$$

with $\boldsymbol{\hat{y}}_k^{(i)}$ defined (copied from above) as

$$\hat{y}_k^{(i)} = \dfrac{\exp(\boldsymbol{w}_k \cdot x^{(i)})}{\displaystyle \sum_{j=1}^K \exp(\boldsymbol{w}_j \cdot x^{(i)})}$$